# 🎙️ Audio Transcription with Google Drive Integration

This notebook allows you to transcribe audio files stored in your Google Drive using the `transcript_pkg`.

## Features:
- 📁 Transcribe single files or entire folders
- 🌐 Support for multiple languages (English, Portuguese, Auto-detect)
- 📝 Multiple output formats (TXT, SRT, VTT)
- 🚀 GPU acceleration support
- 💾 Save results directly to Google Drive


## Tips and Best Practices

### Model Selection Guide:
- **tiny**: Fastest (39M parameters) - Good for quick drafts
- **base**: Balanced (74M parameters) - Recommended for most use cases
- **small**: Better accuracy (244M parameters) - Good for important content
- **medium**: High accuracy (769M parameters) - For professional use
- **large**: Best accuracy (1550M parameters) - When quality is critical

### Performance Tips:
1. **GPU Acceleration**: Google Colab provides free GPU access. Always check GPU is enabled: Runtime → Change runtime type → GPU
2. **Batch Processing**: Process multiple files at once for efficiency
3. **File Size**: For very long audio files (>1 hour), consider splitting them first

### Language Settings:
- Use `auto` for mixed-language content or when unsure
- Use specific language codes (`en`, `pt`) for better accuracy when language is known
- Enable `multilingual` mode for content with multiple languages

### Output Formats:
- **txt**: Plain text, best for reading and searching
- **srt**: Subtitle format with timestamps, compatible with most video players
- **vtt**: WebVTT format, ideal for web-based video players

### Troubleshooting:
- If transcription fails, check file format is supported
- Ensure sufficient Google Drive storage space
- For large batches, monitor Colab runtime limits
- Clear Colab disk space if needed: `!rm -rf /content/*`


## Step 1: Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")


## Step 2: Install Dependencies


In [ ]:
# Install required packages
!pip install -q faster-whisper rich tinytag scipy sounddevice numpy

# Install ffmpeg for audio processing
!apt-get -qq install -y ffmpeg

print("✅ All dependencies installed successfully!")


## Step 3: Setup transcript_pkg


In [ ]:
# Setup transcript_pkg by copying the actual files
# You can either clone your repository or upload the transcript_pkg folder to Colab

import os
import shutil

# Option 1: If you have the transcript_pkg in your repository
# !git clone https://github.com/yourusername/transcriber.git /content/transcriber

# Option 2: Create a minimal version for Colab
os.makedirs('/content/transcript_pkg', exist_ok=True)

# Create __init__.py
with open('/content/transcript_pkg/__init__.py', 'w') as f:
    f.write('"""Audio transcription tools for live and file-based transcription."""\n\n__version__ = "0.1.0"\n')

print("✅ transcript_pkg directory created!")


In [ ]:
# Create a simplified file_transcribe module for Google Colab
file_transcribe_code = '''"""Simplified file transcription module for Google Colab."""

import time
from pathlib import Path
from typing import Optional, List, Dict, Tuple
import numpy as np
import os

from faster_whisper import WhisperModel
from rich.console import Console
from rich.progress import Progress, SpinnerColumn, TextColumn, BarColumn, TaskProgressColumn
from rich.panel import Panel
from rich.table import Table

console = Console()

AUDIO_EXTENSIONS = {
    ".mp3", ".wav", ".flac", ".ogg", ".m4a", ".mp4", 
    ".aac", ".wma", ".opus", ".webm", ".mkv", ".avi", ".mov", ".m4v"
}

LANGUAGE_MAP = {
    "en": "en",
    "pt": "pt", 
    "auto": None
}

def format_timestamp(seconds: float) -> str:
    """Convert seconds to timestamp format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{secs:06.3f}"

def format_duration(seconds: float) -> str:
    """Format duration in human-readable format."""
    if seconds < 60:
        return f"{seconds:.1f}s"
    elif seconds < 3600:
        return f"{seconds/60:.1f}m"
    else:
        return f"{seconds/3600:.1f}h"

class StreamingTranscriptionWriter:
    """Handles streaming output of transcription data."""
    
    def __init__(self, output_file: Path, format_type: str, multilingual: bool = False):
        self.output_file = output_file
        self.format_type = format_type
        self.multilingual = multilingual
        self.segment_count = 0
        self.file_handle = None
        self._initialize_file()
    
    def _initialize_file(self):
        """Initialize the output file with headers if needed."""
        self.output_file.parent.mkdir(parents=True, exist_ok=True)
        
        if self.format_type == "txt":
            self.file_handle = open(self.output_file, "w", encoding="utf-8")
            if self.multilingual:
                self.file_handle.write("[Multilingual transcription - language shown in brackets]\\n\\n")
        elif self.format_type == "vtt":
            self.file_handle = open(self.output_file, "w", encoding="utf-8")
            self.file_handle.write("WEBVTT\\n\\n")
        elif self.format_type == "srt":
            self.file_handle = open(self.output_file, "w", encoding="utf-8")
    
    def write_segment(self, segment):
        """Write a single segment to the output file."""
        if not self.file_handle:
            return
        
        self.segment_count += 1
        
        if self.format_type == "txt":
            if self.multilingual and hasattr(segment, "language"):
                self.file_handle.write(f"[{segment.language}] {segment.text.strip()}\\n")
            else:
                self.file_handle.write(f"{segment.text.strip()}\\n")
        
        elif self.format_type == "srt":
            self.file_handle.write(f"{self.segment_count}\\n")
            start = format_timestamp(segment.start).replace(".", ",")
            end = format_timestamp(segment.end).replace(".", ",")
            self.file_handle.write(f"{start} --> {end}\\n")
            if self.multilingual and hasattr(segment, "language"):
                self.file_handle.write(f"[{segment.language}] {segment.text.strip()}\\n\\n")
            else:
                self.file_handle.write(f"{segment.text.strip()}\\n\\n")
        
        elif self.format_type == "vtt":
            start = format_timestamp(segment.start)
            end = format_timestamp(segment.end)
            self.file_handle.write(f"{start} --> {end}\\n")
            if self.multilingual and hasattr(segment, "language"):
                self.file_handle.write(f"[{segment.language}] {segment.text.strip()}\\n\\n")
            else:
                self.file_handle.write(f"{segment.text.strip()}\\n\\n")
        
        self.file_handle.flush()
    
    def close(self):
        """Close the output file."""
        if self.file_handle:
            self.file_handle.close()

def detect_device_and_compute_type() -> Tuple[str, str]:
    """Detect if GPU is available and return appropriate device and compute type."""
    try:
        import torch
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            console.print(f"[green]✓[/green] GPU detected: {gpu_name}")
            return "cuda", "float16"
    except ImportError:
        pass
    
    console.print("[yellow]ℹ[/yellow] Using CPU for transcription")
    return "cpu", "int8"

def load_whisper_model_with_retry(model_size: str, device: str, compute_type: str, max_retries: int = 3):
    """Load Whisper model with retry logic for handling Hugging Face connectivity issues."""
    import time
    
    for attempt in range(max_retries):
        try:
            console.print(f"[cyan]Loading model (attempt {attempt + 1}/{max_retries})...[/cyan]")
            
            # Try different loading strategies
            try:
                # First try: Normal loading with local_files_only=False
                model = WhisperModel(
                    model_size, 
                    device=device, 
                    compute_type=compute_type,
                    local_files_only=False
                )
                return model
            except Exception as e1:
                console.print(f"[yellow]Standard loading failed: {str(e1)[:100]}...[/yellow]")
                
                # Second try: Download path explicitly
                try:
                    # Set environment variable to avoid the HF token warning
                    os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
                    
                    # Try with download_root
                    download_root = "/content/whisper_models"
                    os.makedirs(download_root, exist_ok=True)
                    
                    model = WhisperModel(
                        model_size,
                        device=device,
                        compute_type=compute_type,
                        download_root=download_root
                    )
                    return model
                except Exception as e2:
                    console.print(f"[yellow]Download root method failed: {str(e2)[:100]}...[/yellow]")
                    
                    # Third try: Use alternative loading
                    if attempt < max_retries - 1:
                        wait_time = (attempt + 1) * 5
                        console.print(f"[yellow]Waiting {wait_time} seconds before retry...[/yellow]")
                        time.sleep(wait_time)
                    else:
                        raise Exception(f"Failed to load model after {max_retries} attempts")
                        
        except Exception as e:
            if attempt == max_retries - 1:
                console.print(f"[red]Failed to load model: {e}[/red]")
                console.print("[yellow]Troubleshooting tips:[/yellow]")
                console.print("1. Try restarting the Colab runtime")
                console.print("2. Check your internet connection")
                console.print("3. Try a different model size (e.g., 'tiny' or 'small')")
                console.print("4. Clear Colab cache: !rm -rf /root/.cache/huggingface")
                raise
    
    return None

def transcribe_file(
    audio_file: Path,
    model: WhisperModel,
    output_dir: Path,
    language: str = "auto",
    output_format: str = "txt",
    multilingual: bool = False
) -> Dict:
    """Transcribe a single audio file."""
    
    # Prepare output file
    output_file = output_dir / f"{audio_file.stem}.{output_format}"
    
    # Create writer
    writer = StreamingTranscriptionWriter(output_file, output_format, multilingual)
    
    # Start transcription
    console.print(f"[cyan]Processing:[/cyan] {audio_file.name}")
    start_time = time.time()
    
    try:
        # Transcribe
        segments, info = model.transcribe(
            str(audio_file),
            language=LANGUAGE_MAP.get(language),
            beam_size=5,
            vad_filter=True,
            vad_parameters=dict(min_silence_duration_ms=500),
            multilingual=multilingual
        )
        
        # Process segments
        segment_count = 0
        for segment in segments:
            writer.write_segment(segment)
            segment_count += 1
        
        # Close writer
        writer.close()
        
        # Calculate stats
        process_time = time.time() - start_time
        speed = info.duration / process_time if process_time > 0 else 0
        
        console.print(
            f"[green]✓[/green] {audio_file.name} "
            f"[dim]({info.duration:.1f}s @ {speed:.1f}x speed, {segment_count} segments)[/dim]"
        )
        
        return {
            "success": True,
            "file": audio_file.name,
            "duration": info.duration,
            "segments": segment_count,
            "process_time": process_time,
            "speed": speed,
            "output_file": str(output_file),
            "detected_language": info.language if language == "auto" else language
        }
        
    except Exception as e:
        writer.close()
        console.print(f"[red]✗[/red] Error processing {audio_file.name}: {e}")
        return {
            "success": False,
            "file": audio_file.name,
            "error": str(e)
        }

def find_audio_files(input_path: Path) -> List[Path]:
    """Find all audio files in the given path."""
    audio_files = []
    
    if input_path.is_file():
        if input_path.suffix.lower() in AUDIO_EXTENSIONS:
            audio_files.append(input_path)
    else:
        for ext in AUDIO_EXTENSIONS:
            audio_files.extend(input_path.glob(f"*{ext}"))
            audio_files.extend(input_path.glob(f"*{ext.upper()}"))
    
    return sorted(audio_files)

def transcribe_folder(
    input_path: Path,
    output_path: Path,
    model_size: str = "base",
    language: str = "auto",
    output_format: str = "txt",
    multilingual: bool = False
) -> List[Dict]:
    """Transcribe all audio files in a folder."""
    
    # Find audio files
    audio_files = find_audio_files(input_path)
    
    if not audio_files:
        console.print("[red]No audio files found![/red]")
        return []
    
    console.print(f"[green]Found {len(audio_files)} audio files[/green]")
    
    # Create output directory
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Load model with retry logic
    device, compute_type = detect_device_and_compute_type()
    console.print(f"[cyan]Loading Whisper {model_size} model...[/cyan]")
    
    try:
        with console.status("[bold cyan]Loading model...[/bold cyan]"):
            model = load_whisper_model_with_retry(model_size, device=device, compute_type=compute_type)
        
        console.print(f"[green]✓[/green] Model loaded successfully!")
    except Exception as e:
        console.print(f"[red]Error loading model: {e}[/red]")
        console.print("\\n[yellow]Alternative solution:[/yellow]")
        console.print("Try running this command first to clear cache:")
        console.print("[cyan]!rm -rf /root/.cache/huggingface[/cyan]")
        console.print("Then restart the runtime and try again.")
        return []
    
    # Process files
    results = []
    with Progress(
        SpinnerColumn(),
        TextColumn("[bold blue]{task.description}"),
        BarColumn(),
        TaskProgressColumn(),
        console=console
    ) as progress:
        task = progress.add_task("Transcribing files...", total=len(audio_files))
        
        for audio_file in audio_files:
            result = transcribe_file(
                audio_file, model, output_path, 
                language, output_format, multilingual
            )
            results.append(result)
            progress.advance(task)
    
    return results

def display_results_summary(results: List[Dict]):
    """Display a summary of transcription results."""
    if not results:
        return
    
    # Create summary table
    table = Table(title="📊 Transcription Results", show_header=True, header_style="bold cyan")
    table.add_column("File", style="cyan", width=30)
    table.add_column("Status", style="green")
    table.add_column("Duration", justify="right")
    table.add_column("Segments", justify="right")
    table.add_column("Speed", justify="right")
    
    successful = 0
    failed = 0
    total_duration = 0
    total_time = 0
    
    for result in results:
        if result["success"]:
            successful += 1
            total_duration += result["duration"]
            total_time += result["process_time"]
            
            table.add_row(
                result["file"][:30] + "..." if len(result["file"]) > 30 else result["file"],
                "✅ Success",
                format_duration(result["duration"]),
                str(result["segments"]),
                f"{result['speed']:.1f}x"
            )
        else:
            failed += 1
            error_msg = str(result["error"])[:20] + "..." if len(str(result["error"])) > 20 else str(result["error"])
            table.add_row(
                result["file"][:30] + "..." if len(result["file"]) > 30 else result["file"],
                f"❌ {error_msg}",
                "-",
                "-",
                "-"
            )
    
    console.print("\\n")
    console.print(table)
    
    # Summary statistics
    if successful > 0:
        avg_speed = total_duration / total_time if total_time > 0 else 0
        console.print("\\n")
        console.print(Panel(
            f"[bold]Summary:[/bold]\\n\\n"
            f"✅ Successful: {successful} files\\n"
            f"❌ Failed: {failed} files\\n"
            f"⏱️  Total audio: {format_duration(total_duration)}\\n"
            f"⚡ Processing time: {format_duration(total_time)}\\n"
            f"🚀 Average speed: {avg_speed:.1f}x realtime",
            title="[bold green]Transcription Complete![/bold green]",
            border_style="green"
        ))
'''

with open('/content/transcript_pkg/file_transcribe.py', 'w') as f:
    f.write(file_transcribe_code)

print("✅ file_transcribe.py created with improved model loading!")


### ⚠️ Troubleshooting: Model Loading Issues

If you encounter a **"502 Bad Gateway"** or similar error when loading the model, this is usually due to Hugging Face connectivity issues. The notebook now includes retry logic to handle this automatically. However, if the issue persists:


In [ ]:
# Clear Hugging Face cache if you encounter model loading errors
# Uncomment and run if needed:
# !rm -rf /root/.cache/huggingface
# !rm -rf /content/whisper_models

# After running this, restart the runtime: Runtime → Restart runtime


## Step 4: Configure Transcription Settings

Modify the parameters below according to your needs:


In [ ]:
# Configuration parameters - MODIFY THESE AS NEEDED

# Input path in Google Drive (can be a file or folder)
INPUT_PATH = "/content/drive/MyDrive/AudioFiles"  # @param {type:"string"}

# Output path in Google Drive  
OUTPUT_PATH = "/content/drive/MyDrive/Transcriptions"  # @param {type:"string"}

# Model size: tiny, base, small, medium, large
MODEL_SIZE = "base"  # @param ["tiny", "base", "small", "medium", "large"]

# Language: en (English), pt (Portuguese), auto (auto-detect)
LANGUAGE = "auto"  # @param ["auto", "en", "pt"]

# Output format: txt, srt, vtt
OUTPUT_FORMAT = "txt"  # @param ["txt", "srt", "vtt"]

# Enable multilingual mode (shows language for each segment)
MULTILINGUAL = False  # @param {type:"boolean"}

print("📋 Configuration:")
print(f"  Input: {INPUT_PATH}")
print(f"  Output: {OUTPUT_PATH}")
print(f"  Model: {MODEL_SIZE}")
print(f"  Language: {LANGUAGE}")
print(f"  Format: {OUTPUT_FORMAT}")
print(f"  Multilingual: {MULTILINGUAL}")


## Step 5: Run Transcription

Execute the cell below to start the transcription process:


In [ ]:
from pathlib import Path
import sys
sys.path.append('/content')

from transcript_pkg.file_transcribe import transcribe_folder, display_results_summary
from rich.console import Console
from rich.panel import Panel

console = Console()

# Convert paths
input_path = Path(INPUT_PATH)
output_path = Path(OUTPUT_PATH)

# Check if input exists
if not input_path.exists():
    console.print(f"[red]Error: Input path does not exist: {INPUT_PATH}[/red]")
    console.print("[yellow]Please check your INPUT_PATH and ensure the folder/file exists in your Google Drive.[/yellow]")
else:
    # Run transcription
    console.print(Panel(
        "[bold green]Starting Transcription Process[/bold green]\n\n"
        "This may take a while depending on the size and number of files.\n"
        "The process will use GPU acceleration if available.",
        title="🎙️ Transcription",
        border_style="green"
    ))
    
    results = transcribe_folder(
        input_path=input_path,
        output_path=output_path,
        model_size=MODEL_SIZE,
        language=LANGUAGE,
        output_format=OUTPUT_FORMAT,
        multilingual=MULTILINGUAL
    )
    
    # Display results summary
    if results:
        display_results_summary(results)
        
        # Show output location
        console.print(f"\n📁 [bold cyan]Output files saved to:[/bold cyan] {output_path}")
        console.print("[dim]You can find your transcriptions in the specified Google Drive folder.[/dim]")


## Step 6: View Transcription Files

List and preview the transcribed files:


In [ ]:
# List all transcription files created
import os
from rich.table import Table

if output_path.exists():
    files = list(output_path.glob(f"*.{OUTPUT_FORMAT}"))
    
    if files:
        # Create a table to display files
        table = Table(title=f"📄 Transcription Files ({len(files)} total)", show_header=True)
        table.add_column("File Name", style="cyan")
        table.add_column("Size", justify="right", style="yellow")
        table.add_column("Path", style="dim")
        
        for file in sorted(files):
            size_kb = os.path.getsize(file) / 1024
            table.add_row(
                file.name,
                f"{size_kb:.1f} KB",
                str(file.relative_to(Path("/content/drive")))
            )
        
        console.print(table)
    else:
        console.print("[yellow]No transcription files found in the output directory.[/yellow]")
else:
    console.print("[red]Output directory does not exist.[/red]")


## Step 7: Preview a Transcription

Preview the content of the first transcription file:


In [ ]:
# Preview a transcription file
if output_path.exists():
    files = list(output_path.glob(f"*.{OUTPUT_FORMAT}"))
    
    if files:
        # Let user select which file to preview
        console.print(f"\n[cyan]Found {len(files)} transcription file(s). Showing preview of the first one.[/cyan]")
        
        # Get the first file
        preview_file = sorted(files)[0]
        
        console.print(f"\n[bold]File: {preview_file.name}[/bold]\n")
        
        # Read and display content (limit to first 1000 characters for preview)
        try:
            with open(preview_file, 'r', encoding='utf-8') as f:
                content = f.read()
                preview_length = 1000
                
                if len(content) > preview_length:
                    preview = content[:preview_length] + "\n\n[... truncated for preview ...]"
                else:
                    preview = content
                
                console.print(Panel(
                    preview,
                    title=f"📄 {preview_file.name}",
                    border_style="blue",
                    padding=(1, 2)
                ))
                
                console.print(f"\n[dim]Full file location: {preview_file}[/dim]")
                console.print(f"[dim]Total content length: {len(content)} characters[/dim]")
                
        except Exception as e:
            console.print(f"[red]Error reading file: {e}[/red]")
    else:
        console.print("[yellow]No transcription files found to preview.[/yellow]")
else:
    console.print("[red]Output directory does not exist.[/red]")
